In [1]:
import sympy as sp
from sympy.abc import symbols
from sympy import init_printing

In [2]:
r_x, r_y, r_z = symbols("r_x, r_y, r_z", real=True)
mu_x, mu_y, mu_z = symbols("mu_x, mu_y, mu_z", real=True)
t_x, t_y, t_z = symbols("t_x, t_y, t_z", real=True)

In [3]:
c_x, c_y, f_x, f_y = symbols("c_x, c_y, f_x, f_y", positive=True, real=True)
u, v = symbols("u, v", real=True)

r_pinhole = sp.Matrix([
    (u - c_x) / f_x,
    (v - c_y) / f_y,
    1
])

fisheye_radius = r_pinhole[:2, :].norm()

r = sp.Matrix([
    (u - c_x) * sp.sin(fisheye_radius) / (f_x * fisheye_radius),
    (v - c_y) * sp.sin(fisheye_radius) / (f_y * fisheye_radius),
    sp.cos(fisheye_radius)
])

r

Matrix([
[(-c_x + u)*sin(sqrt((c_y - v)**2/f_y**2 + (c_x - u)**2/f_x**2))/(f_x*sqrt((c_y - v)**2/f_y**2 + (c_x - u)**2/f_x**2))],
[(-c_y + v)*sin(sqrt((c_y - v)**2/f_y**2 + (c_x - u)**2/f_x**2))/(f_y*sqrt((c_y - v)**2/f_y**2 + (c_x - u)**2/f_x**2))],
[                                                                 cos(sqrt((c_y - v)**2/f_y**2 + (c_x - u)**2/f_x**2))]])

In [4]:
mu_vec = sp.Matrix([mu_x, mu_y, mu_z])
t = sp.Matrix([t_x, t_y, t_z])

In [5]:
r_vec = sp.Matrix([r_x, r_y, r_z])
mu = r_vec / r_vec.norm()
mu

Matrix([
[r_x/sqrt(r_x**2 + r_y**2 + r_z**2)],
[r_y/sqrt(r_x**2 + r_y**2 + r_z**2)],
[r_z/sqrt(r_x**2 + r_y**2 + r_z**2)]])

In [38]:
a = sp.simplify(mu.subs(r_x, r[0]).subs(r_y, r[1]).subs(r_z, r[2]))
a
# sp.simplify(a.norm().subs(sp.Abs, sp.Id))

Matrix([
[-f_y*(c_x - u)*sin(sqrt(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)/(f_x*f_y))/sqrt(c_x**2*f_y**2 - 2*c_x*f_y**2*u + c_y**2*f_x**2 - 2*c_y*f_x**2*v + f_x**2*v**2 + f_y**2*u**2)],
[-f_x*(c_y - v)*sin(sqrt(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)/(f_x*f_y))/sqrt(c_x**2*f_y**2 - 2*c_x*f_y**2*u + c_y**2*f_x**2 - 2*c_y*f_x**2*v + f_x**2*v**2 + f_y**2*u**2)],
[                                                               cos(sqrt(c_x**2*f_y**2 - 2*c_x*f_y**2*u + c_y**2*f_x**2 - 2*c_y*f_x**2*v + f_x**2*v**2 + f_y**2*u**2)/(f_x*f_y))]])

In [6]:
x_2d = t / t.dot(mu_vec)
x_2d

Matrix([
[t_x/(mu_x*t_x + mu_y*t_y + mu_z*t_z)],
[t_y/(mu_x*t_x + mu_y*t_y + mu_z*t_z)],
[t_z/(mu_x*t_x + mu_y*t_y + mu_z*t_z)]])

In [7]:
mu_xz_vec = sp.Matrix([mu_x, mu_z])
mu_xz_norm = mu_xz_vec.norm()
mu_norm = mu.norm()

Q = sp.Matrix([
    [mu_z / mu_xz_norm, 0, -mu_x / mu_xz_norm],
    [-(mu_x * mu_y)/(mu_xz_norm * mu_norm), mu_xz_norm / mu_norm, -(mu_y * mu_z)/(mu_xz_norm * mu_norm)],
    [mu_x / mu_norm, mu_y / mu_norm, mu_z / mu_norm]
])
Q

Matrix([
[                                                                                                                        mu_z/sqrt(mu_x**2 + mu_z**2),                                                                                                                                       0,                                                                                                                        -mu_x/sqrt(mu_x**2 + mu_z**2)],
[-mu_x*mu_y/(sqrt(mu_x**2 + mu_z**2)*sqrt(r_x**2/(r_x**2 + r_y**2 + r_z**2) + r_y**2/(r_x**2 + r_y**2 + r_z**2) + r_z**2/(r_x**2 + r_y**2 + r_z**2))), sqrt(mu_x**2 + mu_z**2)/sqrt(r_x**2/(r_x**2 + r_y**2 + r_z**2) + r_y**2/(r_x**2 + r_y**2 + r_z**2) + r_z**2/(r_x**2 + r_y**2 + r_z**2)), -mu_y*mu_z/(sqrt(mu_x**2 + mu_z**2)*sqrt(r_x**2/(r_x**2 + r_y**2 + r_z**2) + r_y**2/(r_x**2 + r_y**2 + r_z**2) + r_z**2/(r_x**2 + r_y**2 + r_z**2)))],
[                                mu_x/sqrt(r_x**2/(r_x**2 + r_y**2 + r_z**2) + r_y**2/(r_x**2 + r_y**2 + r_z**2) 

In [8]:
d = Q @ x_2d
d.simplify()
d

Matrix([
[                                 (-mu_x*t_z + mu_z*t_x)/(sqrt(mu_x**2 + mu_z**2)*(mu_x*t_x + mu_y*t_y + mu_z*t_z))],
[(-mu_y*(mu_x*t_x + mu_z*t_z) + t_y*(mu_x**2 + mu_z**2))/(sqrt(mu_x**2 + mu_z**2)*(mu_x*t_x + mu_y*t_y + mu_z*t_z))],
[                                                                                                                 1]])

$\displaystyle \left[\begin{matrix}\frac{- \mu_{x} t_{z} + \mu_{z} t_{x}}{\sqrt{\mu_{x}^{2} + \mu_{z}^{2}} \left(\mu_{x} t_{x} + \mu_{y} t_{y} + \mu_{z} t_{z}\right)}\\\frac{- \mu_{y} \left(\mu_{x} t_{x} + \mu_{z} t_{z}\right) + t_{y} \left(\mu_{x}^{2} + \mu_{z}^{2}\right)}{\sqrt{\mu_{x}^{2} + \mu_{z}^{2}} \left(\mu_{x} t_{x} + \mu_{y} t_{y} + \mu_{z} t_{z}\right)}\\1\end{matrix}\right]$

In [9]:
d_simple = sp.simplify(d.subs(mu_x, mu[0]).subs(mu_y, mu[1]).subs(mu_z, mu[2]))
# d_simple_2 = sp.simplify(d_simple.subs(r_x, r[0]).subs(r_y, r[1]).subs(r_z, r[2]))
d_simple

Matrix([
[(-r_x*t_z + r_z*t_x)*sqrt(r_x**2 + r_y**2 + r_z**2)/(sqrt(r_x**2 + r_z**2)*(r_x*t_x + r_y*t_y + r_z*t_z))],
[ (-r_y*(r_x*t_x + r_z*t_z) + t_y*(r_x**2 + r_z**2))/(sqrt(r_x**2 + r_z**2)*(r_x*t_x + r_y*t_y + r_z*t_z))],
[                                                                                                        1]])

In [10]:
def jacobian(a: sp.Matrix, b: sp.Matrix):
    rows, cols = a.shape[0], b.shape[0]
    J = sp.zeros(rows, cols)
    for r in range(rows):
        for c in range(cols):
            J[r, c] = sp.diff(a[r], b[c]).simplify()

    return J

In [11]:
deld_delmu = jacobian(d[0:2, :], mu_vec)
deld_delmu

Matrix([
[                                                                                      (mu_x*(mu_x*t_z - mu_z*t_x)*(mu_x*t_x + mu_y*t_y + mu_z*t_z) + t_x*(mu_x**2 + mu_z**2)*(mu_x*t_z - mu_z*t_x) - t_z*(mu_x**2 + mu_z**2)*(mu_x*t_x + mu_y*t_y + mu_z*t_z))/((mu_x**2 + mu_z**2)**(3/2)*(mu_x*t_x + mu_y*t_y + mu_z*t_z)**2),                                                                                             t_y*(mu_x*t_z - mu_z*t_x)/(sqrt(mu_x**2 + mu_z**2)*(mu_x*t_x + mu_y*t_y + mu_z*t_z)**2),                                                                                        (mu_z*(mu_x*t_z - mu_z*t_x)*(mu_x*t_x + mu_y*t_y + mu_z*t_z) + t_x*(mu_x**2 + mu_z**2)*(mu_x*t_x + mu_y*t_y + mu_z*t_z) + t_z*(mu_x**2 + mu_z**2)*(mu_x*t_z - mu_z*t_x))/((mu_x**2 + mu_z**2)**(3/2)*(mu_x*t_x + mu_y*t_y + mu_z*t_z)**2)],
[(mu_x*(mu_y*(mu_x*t_x + mu_z*t_z) - t_y*(mu_x**2 + mu_z**2))*(mu_x*t_x + mu_y*t_y + mu_z*t_z) + t_x*(mu_x**2 + mu_z**2)*(mu_y*(mu_x*t_x + mu_z*t_z) - t_y*(mu_x**2 +

In [12]:
delmu_delr = jacobian(mu, r_vec)
delmu_delr

Matrix([
[(r_y**2 + r_z**2)/(r_x**2 + r_y**2 + r_z**2)**(3/2),          -r_x*r_y/(r_x**2 + r_y**2 + r_z**2)**(3/2),          -r_x*r_z/(r_x**2 + r_y**2 + r_z**2)**(3/2)],
[         -r_x*r_y/(r_x**2 + r_y**2 + r_z**2)**(3/2), (r_x**2 + r_z**2)/(r_x**2 + r_y**2 + r_z**2)**(3/2),          -r_y*r_z/(r_x**2 + r_y**2 + r_z**2)**(3/2)],
[         -r_x*r_z/(r_x**2 + r_y**2 + r_z**2)**(3/2),          -r_y*r_z/(r_x**2 + r_y**2 + r_z**2)**(3/2), (r_x**2 + r_y**2)/(r_x**2 + r_y**2 + r_z**2)**(3/2)]])

In [13]:
deld_delr = deld_delmu @ delmu_delr
deld_delr.simplify()
deld_delr

Matrix([
[                                                                                                                                                                                                                                                                        (-r_x*r_y*t_y*(mu_x**2 + mu_z**2)*(mu_x*t_z - mu_z*t_x) - r_x*r_z*(mu_z*(mu_x*t_z - mu_z*t_x)*(mu_x*t_x + mu_y*t_y + mu_z*t_z) + t_x*(mu_x**2 + mu_z**2)*(mu_x*t_x + mu_y*t_y + mu_z*t_z) + t_z*(mu_x**2 + mu_z**2)*(mu_x*t_z - mu_z*t_x)) + (r_y**2 + r_z**2)*(mu_x*(mu_x*t_z - mu_z*t_x)*(mu_x*t_x + mu_y*t_y + mu_z*t_z) + t_x*(mu_x**2 + mu_z**2)*(mu_x*t_z - mu_z*t_x) - t_z*(mu_x**2 + mu_z**2)*(mu_x*t_x + mu_y*t_y + mu_z*t_z)))/((mu_x**2 + mu_z**2)**(3/2)*(r_x**2 + r_y**2 + r_z**2)**(3/2)*(mu_x*t_x + mu_y*t_y + mu_z*t_z)**2),                                                                                                                                                                                                          

In [17]:
deld_delr_simple = deld_delr.copy()
deld_delr_simple = deld_delr_simple.subs(mu_x, mu[0]).subs(mu_y, mu[1]).subs(mu_z, mu[2])
deld_delr_simple.simplify()
deld_delr_simple

Matrix([
[                                                           (-r_x*r_y*t_y*(r_x**2 + r_z**2)*(r_x*t_z - r_z*t_x) - r_x*r_z*(r_z*(r_x*t_z - r_z*t_x)*(r_x*t_x + r_y*t_y + r_z*t_z) + t_x*(r_x**2 + r_z**2)*(r_x*t_x + r_y*t_y + r_z*t_z) + t_z*(r_x**2 + r_z**2)*(r_x*t_z - r_z*t_x)) + (r_y**2 + r_z**2)*(r_x*(r_x*t_z - r_z*t_x)*(r_x*t_x + r_y*t_y + r_z*t_z) + t_x*(r_x**2 + r_z**2)*(r_x*t_z - r_z*t_x) - t_z*(r_x**2 + r_z**2)*(r_x*t_x + r_y*t_y + r_z*t_z)))/((r_x**2 + r_z**2)**(3/2)*sqrt(r_x**2 + r_y**2 + r_z**2)*(r_x*t_x + r_y*t_y + r_z*t_z)**2), (r_x**3*t_y*t_z - r_x**2*r_y*t_x*t_z - r_x**2*r_z*t_x*t_y + r_x*r_y*r_z*t_x**2 - r_x*r_y*r_z*t_z**2 + r_x*r_z**2*t_y*t_z + r_y*r_z**2*t_x*t_z - r_z**3*t_x*t_y)/(sqrt(r_x**2 + r_z**2)*sqrt(r_x**2 + r_y**2 + r_z**2)*(r_x**2*t_x**2 + 2*r_x*r_y*t_x*t_y + 2*r_x*r_z*t_x*t_z + r_y**2*t_y**2 + 2*r_y*r_z*t_y*t_z + r_z**2*t_z**2)),                                                             (-r_x*r_z*(r_x*(r_x*t_z - r_z*t_x)*(r_x*t_x + r_y*t_y + r_z*t_z)

In [22]:
sp.cse(deld_delr_simple, optimizations="basic")[1][0]

Matrix([
[                                                                                 -x19*(r_x*x7 + x14*x15 - x17*(x16 + x4)), x18*x41*(x20*x21 - x22*x23 + x24*x25 - x25*x29 + x26*x4 - x28*x3 + x28*x4 - x3*x30),                                                                                   -x19*(r_z*x7 - x14*(x16 + x3) + x15*x17)],
[x51*(r_x*x10*x45 + t_y*x43 + 2*x10*x27*x3 - x16*x48 + x22*x26 - x23*x42 + x24*x44 - x3*x48 + x32*x44 + x39*x47 + x46*x47),                                                              -x41*(x40 + x46 + x52), x51*(r_y*t_z*x34*x4 + r_z*x45*x9 + t_y*x50 - x16*x55 + x20*x30 - x21*x49 + x29*x53 + x32*x53 + x38*x54 - x4*x55 + x52*x54)]])

In [14]:
delr_deluv = jacobian(r, sp.Matrix([u, v]))
delr_deluv

Matrix([
[-f_y**3*(c_x - u)**2*sin(sqrt(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)/(f_x*f_y))/(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)**(3/2) + f_y**2*(c_x - u)**2*cos(sqrt(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)/(f_x*f_y))/(f_x**3*(c_y - v)**2 + f_x*f_y**2*(c_x - u)**2) + f_y*sin(sqrt(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)/(f_x*f_y))/sqrt(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2),                                                                             f_x*(c_x - u)*(c_y - v)*(-f_x*f_y*(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)*sin(sqrt(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)/(f_x*f_y)) + (f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)**(3/2)*cos(sqrt(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)/(f_x*f_y)))/(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)**(5/2)],
[                                                                            f_y*(c_x - u)*(c_y - v)*(-f_x*f_y*(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)*sin(sqrt(f_x**2*(c_y - v)**2 + f_y**2*(c_x - u)**2)/(f_x*f_y))

In [15]:
sp.cse(delr_deluv)[1][0]

Matrix([
[x13 + x14*x5/(f_x*x5 + x15) - x16*x18,                               f_x*x19],
[                              f_y*x19, x14*x2/(f_y*x2 + x16) - x15*x18 + x20],
[                            x13*x3*x8,                             x0*x20*x9]])

In [26]:
deld_deluv = deld_delr_simple @ delr_deluv

In [30]:
sp.cse(deld_deluv)[1][0]

Matrix([
[x14*x35 + x47*x62 + x66*x67, x35*x69 + x47*x70 + x67*x71],
[x14*x84 + x62*x74 + x66*x88, x69*x84 + x70*x74 + x71*x88]])

In [ ]:
a = deld_deluv[0, 0].subs(mu_x, mu[0]).subs(mu_y, mu[1]).subs(mu_z, mu[2]).simplify()

In [26]:
b = a.subs(r_z, r[2]).simplify()
b

(-r_x*r_y*t_y*(r_x**2 + 1)*(r_x*t_z - t_x) - r_x*(t_x*(r_x**2 + 1)*(r_x*t_x + r_y*t_y + t_z) + t_z*(r_x**2 + 1)*(r_x*t_z - t_x) + (r_x*t_z - t_x)*(r_x*t_x + r_y*t_y + t_z)) + (r_y**2 + 1)*(r_x*(r_x*t_z - t_x)*(r_x*t_x + r_y*t_y + t_z) + t_x*(r_x**2 + 1)*(r_x*t_z - t_x) - t_z*(r_x**2 + 1)*(r_x*t_x + r_y*t_y + t_z)))/(f_x*(r_x**2 + 1)**(3/2)*sqrt(r_x**2 + r_y**2 + 1)*(r_x*t_x + r_y*t_y + t_z)**2)

In [32]:
deld_deluv_simple = deld_deluv.copy()
deld_deluv_simple = deld_deluv_simple.subs(mu_x, mu[0]).subs(mu_y, mu[1]).subs(mu_z, mu[2]).subs(r_z, r[2])
deld_deluv_simple.simplify()
deld_deluv_simple

Matrix([
[                               (-r_x*r_y*t_y*(r_x**2 + 1)*(r_x*t_z - t_x) - r_x*(t_x*(r_x**2 + 1)*(r_x*t_x + r_y*t_y + t_z) + t_z*(r_x**2 + 1)*(r_x*t_z - t_x) + (r_x*t_z - t_x)*(r_x*t_x + r_y*t_y + t_z)) + (r_y**2 + 1)*(r_x*(r_x*t_z - t_x)*(r_x*t_x + r_y*t_y + t_z) + t_x*(r_x**2 + 1)*(r_x*t_z - t_x) - t_z*(r_x**2 + 1)*(r_x*t_x + r_y*t_y + t_z)))/(f_x*(r_x**2 + 1)**(3/2)*sqrt(r_x**2 + r_y**2 + 1)*(r_x*t_x + r_y*t_y + t_z)**2), (r_x**3*t_y*t_z - r_x**2*r_y*t_x*t_z - r_x**2*t_x*t_y + r_x*r_y*t_x**2 - r_x*r_y*t_z**2 + r_x*t_y*t_z + r_y*t_x*t_z - t_x*t_y)/(f_y*sqrt(r_x**2 + 1)*sqrt(r_x**2 + r_y**2 + 1)*(r_x**2*t_x**2 + 2*r_x*r_y*t_x*t_y + 2*r_x*t_x*t_z + r_y**2*t_y**2 + 2*r_y*t_y*t_z + t_z**2))],
[(r_x**3*r_y*t_x**2 + r_x**3*r_y*t_y**2 + r_x**3*t_y*t_z + 2*r_x**2*r_y*t_x*t_z - r_x**2*t_x*t_y + r_x*r_y**2*t_y*t_z + r_x*r_y*t_y**2 + r_x*r_y*t_z**2 + r_x*t_y*t_z - r_y**2*t_x*t_y - t_x*t_y)/(f_x*sqrt(r_x**2 + 1)*(r_x**4*t_x**2 + 2*r_x**3*r_y*t_x*t_y + 2*r_x**3*t_x*t_z + r_x**2*r_y**2*

In [35]:
variables, exp = sp.cse(deld_deluv_simple)

In [38]:
variables

[(x0, 1/f_x),
 (x1, r_x**2),
 (x2, x1 + 1),
 (x3, r_y**2),
 (x4, 1/sqrt(x2 + x3)),
 (x5, r_x*t_x),
 (x6, r_y*t_y),
 (x7, t_z + x5 + x6),
 (x8, r_x*t_z),
 (x9, -t_x + x8),
 (x10, t_z*x2),
 (x11, t_x*x2),
 (x12, x7*x9),
 (x13, t_x**2),
 (x14, r_x*r_y),
 (x15, t_x*t_z),
 (x16, r_y*x15),
 (x17, t_z**2),
 (x18, x14*x17),
 (x19, x1*x16),
 (x20, t_x*t_y),
 (x21, t_y*x8),
 (x22, r_x**3),
 (x23, t_y*t_z*x22 - x1*x20 - x20 + x21),
 (x24, 1/sqrt(x2)),
 (x25, t_y**2),
 (x26, x25*x3),
 (x27, 2*t_z),
 (x28, x27*x6),
 (x29, x1*x13 + x17 + x27*x5),
 (x30, x26 + x28 + x29 + 2*x5*x6),
 (x31, x24/(f_y*x30)),
 (x32, r_y*x22),
 (x33, 2*x22)]

In [37]:
exp[0]

Matrix([
[                                   x0*x4*(-r_x*x2*x6*x9 - r_x*(x10*x9 + x11*x7 + x12) + (x3 + 1)*(r_x*x12 - x10*x7 + x11*x9))/(x2**(3/2)*x7**2), x31*x4*(x13*x14 + x16 - x18 - x19 + x23)],
[x0*x24*(x13*x32 + x14*x25 + x18 + 2*x19 - x20*x3 + x21*x3 + x23 + x25*x32)/(r_x**4*x13 + t_x*x33*x6 + x1*x17 + x1*x26 + x1*x28 + x15*x33 + x30),                x31*(-x1*x25 - x25 - x29)]])